In [30]:
import os
import re
import csv
import statistics
import pandas as pd
import numpy as np
import geopandas as gpd
from shapely.geometry import Point, Polygon

In [31]:
# test files for one participant and one gaze file
eye_files = os.listdir('../../../../code_summ_data/168/gaze')
eye_files = os.listdir('../../../../code_summ_data/182/gaze')
#eye_filename = '../../../../code_summ_data/168/gaze/168_gaze_reading_createServerChooser.csv'
#eye_filename = '../../../../code_summ_data/168/gaze/168_gaze_writing_BFSdist.csv'
eye_filename = '../../../../code_summ_data/182/gaze/182_gaze_reading_addDigit.csv'
eye_file = pd.read_csv(eye_filename)

In [32]:
# resetting column names
print(eye_file.shape)
num_cols = len(eye_file.columns)
if num_cols == 13:
    eye_file.columns = ['participant_id', 'function_name', 'function_id', 'system_timestamp', 'device_timestamp', 'valid_gaze_left',
                        'valid_gaze_right', 'gaze_left_eye', 'gaze_right_eye', 'valid_pd_left', 'valid_pd_right', 'pd_left', 'pd_right']
elif num_cols == 17:
    eye_file.columns = ['participant_id', 'function_name', 'function_fid', 'system_timestamp', 'device_timestamp', 'valid_gaze_left', 'valid_gaze_right', 'gaze_left_eye', 'gaze_right_eye',
                         'valid_pd_left', 'valid_pd_right', 'pd_left', 'pd_right', 'irl_left_eye_coordinates', 'irl_right_eye_coordinates', 'irl_left_point_on_screen', 'irl_right_point_on_screen']


(11897, 17)


In [17]:
height = 1080
width = 1920


def normalize_coordinates(x, y, w, h):
    x /= width
    y /= height
    w /= width
    h /= height
    return x, y, w, h


In [36]:
#ref = pd.read_csv('./word_coordinates_final/createServerChooser_boxes.csv') # reading in bounding boxes
ref = pd.read_csv('./word_coordinates_final/addDigit_boxes.csv')
boxes = pd.DataFrame()

for i in range(len(ref)): # creating shapes for each word in the file
    x = ref.loc[i, 'tobii_x']
    y = ref.loc[i, 'tobii_y']
    w = ref.loc[i, 'tobii_width']
    h = ref.loc[i, 'tobii_height']
    word = '{word}.{num}'.format(word=ref.loc[i, 'word'], num=ref.loc[i, 'occurrence'])
    # Creating a shape for each word based on its coordinates
    new_tangle = pd.Series({word: Polygon([(x, y), (x+w, y), (x, y+h), (x+w, y+h)])}) 
    boxes = pd.concat([boxes, new_tangle]) # adding this to a new data structure 

In [37]:
aois = [(1175, 200, 700, 125), # prewritten summary
        (1175, 450, 600, 100), # 'accurate' question
        (1175, 550, 600, 150), # 'missing' question
        (1175, 725, 600, 100), # 'unnecessary' question
        (1175, 825, 600, 125), # 'readable' question
        (1165, 85, 720, 300)]  # participant summary

tobii_aois = []
aoi_names = ['prewritten', 'accurate', 'missing', 'unnecessary', 'readable', 'participant_summary']
for box in aois:
        x, y, w, h = normalize_coordinates(box[0], box[1], box[2], box[3])
        tobii_aois.append((x, y, w, h))
print(len(tobii_aois))
# x, y, w, h = normalize_coordinates(1175, 450, 600, 100)
# print(x, y, w, h)

6


In [38]:
if re.search('reading', eye_filename):
    for i in range(5):
        x = tobii_aois[i][0] 
        y = tobii_aois[i][1]
        w = tobii_aois[i][2]
        h = tobii_aois[i][3]
        new_tangle = pd.Series({aoi_names[i]: Polygon([(x, y), (x+w, y), (x, y+h), (x+w, y+h)])})
        boxes = pd.concat([boxes, new_tangle])
        
elif re.search('writing', eye_filename):
    x = tobii_aois[5][0]
    y = tobii_aois[5][1]
    w = tobii_aois[5][2]
    h = tobii_aois[5][3]
    new_tangle = pd.Series({"participant_summary": Polygon([(x, y), (x+w, y), (x, y+h), (x+w, y+h)])})
    boxes = pd.concat([boxes, new_tangle])
    
    
gpd_boxes = gpd.GeoSeries(boxes[0]) # data structure of boxes

In [39]:
print(boxes)

                                                                0
private.0       POLYGON ((0.0114583333333333 0.125, 0.05312499...
void.0          POLYGON ((0.0557291666666666 0.125, 0.08072916...
addDigit.0      POLYGON ((0.084375 0.125, 0.1359375 0.125, 0.0...
char.0          POLYGON ((0.140625 0.125, 0.1651041666666666 0...
digit.0         POLYGON ((0.16875 0.125, 0.1989583333333333 0....
if.0            POLYGON ((0.0333333333333333 0.15, 0.047395833...
digit.1         POLYGON ((0.0614583333333333 0.15, 0.091666666...
==.0            POLYGON ((0.0953125 0.15, 0.109375 0.15, 0.095...
0'.0            POLYGON ((0.1140625 0.15, 0.1302083333333333 0...
&&.0            POLYGON ((0.134375 0.15, 0.1489583333333333 0....
value.equals.0  POLYGON ((0.1515625 0.15, 0.2260416666666666 0...
"0".0           POLYGON ((0.23125 0.15, 0.2489583333333333 0.1...
return.0        POLYGON ((0.2703125 0.15, 0.3104166666666666 0...
if.1            POLYGON ((0.0333333333333333 0.175, 0.04739583...
value.equa

In [40]:
#newfile = '168_localizing_gaze_test_createServerChooser.csv'
newfile = '182_localizing_gaze_test_reading_addDigit.csv'
new_eye_df = pd.DataFrame()

for i in range(len(eye_file)): # iterating through eyetracking file and localizing gaze to a shape
    gaze_left = eye_file.loc[i, 'gaze_left_eye']
    gaze_right = eye_file.loc[i, 'gaze_right_eye']
    temp_left = re.split(r'[(,\)]', gaze_left) # tuple is a string, so using regex to get the numbers
    temp_right = re.split(r'[(,\)]', gaze_right)
    # averaging right and left gaze points for one gaze point
    gaze_point = (statistics.fmean([float(temp_left[1]), float(temp_right[1])]), 
                 statistics.fmean([float(temp_left[2]), float(temp_right[2])]))
    if pd.isna(gaze_point[0]) or pd.isna(gaze_point[1]): # if either eye is invalid
        continue
    else:
        gaze_point = [Point(gaze_point[0], gaze_point[1])]
    pnts = gpd.GeoDataFrame(geometry=gaze_point)
    
    # confusing, but basically this below line returns false/true whether the point is in a shape
    pnts = pnts.assign(**{key: pnts.within(geom) for key, geom in gpd_boxes.items()}) 
    temp = pd.DataFrame(pnts)
    temp = temp.replace({True : 1, False : ''})
    #temp.to_csv('168_localizing_gaze_test_createServerChooser.csv')
    old_row = eye_file.loc[i, :]
    new_row = pd.concat([old_row, temp.T]) # adding this true/false information to the original row
    new_eye_df = pd.concat([new_eye_df, new_row], axis=1)
new_eye_df = new_eye_df.T
new_eye_df.to_csv(newfile)

In [16]:
gpd_boxes = gpd.GeoDataFrame({'geometry': boxes[0]})

# Create a spatial index
sindex = gpd_boxes.sindex

# Process the eye_file DataFrame to extract gaze points
gaze_points = []
for i, row in eye_file.iterrows():
    gaze_left = row['gaze_left_eye']
    gaze_right = row['gaze_right_eye']
    temp_left = re.split(r'[(,\)]', gaze_left)
    temp_right = re.split(r'[(,\)]', gaze_right)
    gaze_point = (statistics.fmean([float(temp_left[1]), float(temp_right[1])]),
                  statistics.fmean([float(temp_left[2]), float(temp_right[2])]))
    if not pd.isna(gaze_point[0]) and not pd.isna(gaze_point[1]):
        gaze_points.append(Point(gaze_point[0], gaze_point[1]))

# Create a GeoDataFrame with gaze points
gpd_gaze_points = gpd.GeoDataFrame(geometry=gaze_points)

# Find the boxes containing each gaze point
gpd_gaze_points['box_index'] = -1  # Initialize with -1 (not found)
for i, gaze_point in gpd_gaze_points.iterrows():
    intersecting_boxes = list(sindex.intersection(gaze_point.geometry.bounds))
    for box_idx in intersecting_boxes:
        if gpd_boxes.loc[box_idx, 'geometry'].contains(gaze_point.geometry):
            gpd_gaze_points.loc[i, 'box_index'] = box_idx
            break

# Now you can merge the gpd_gaze_points GeoDataFrame with your original DataFrame (eye_file)
# on the index, and the 'box_index' column in gpd_gaze_points will show the box index for each gaze point
new_eye_df = eye_file.merge(
    gpd_gaze_points[['box_index']], left_index=True, right_index=True)
new_eye_df.to_csv(newfile)


KeyError: 2